## Inital Data Upload and EDA

In this notebook, we will upload and begin to explore our dataset(s). First, let's set up our standard modules.

In [46]:
# Import standard data science & visualization packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Our first dataset comes from [Moneypuck.com](moneypuck.com). It contains Advanced Stats for all NHL games from the 2008-09 season to the 2022-23 season, aggregated at a team level.

In [47]:
mp_df = pd.read_csv('all_teams.csv')

In [4]:
mp_df.shape

(190300, 111)

In [5]:
# .info won't show the individual column details unless we include the number of columns to show. There are 111 columns in this dataset.
mp_df.info(111)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190300 entries, 0 to 190299
Data columns (total 111 columns):
 #    Column                                     Dtype  
---   ------                                     -----  
 0    team                                       object 
 1    season                                     int64  
 2    name                                       object 
 3    gameId                                     int64  
 4    playerTeam                                 object 
 5    opposingTeam                               object 
 6    home_or_away                               object 
 7    gameDate                                   int64  
 8    position                                   object 
 9    situation                                  object 
 10   xGoalsPercentage                           float64
 11   corsiPercentage                            float64
 12   fenwickPercentage                          float64
 13   iceTime                    

In [6]:
mp_df.head()

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,...,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame
0,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,other,...,1.0,1.000,0.0,0.017,0.000,0.000,0.037,0.037,0.037,0
1,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,all,...,31.0,30.369,5.0,0.396,0.168,0.168,2.917,2.833,2.714,0
2,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,5on5,...,20.0,19.369,3.0,0.237,0.168,0.168,1.862,1.777,1.665,0
3,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,4on5,...,9.0,9.000,1.0,0.124,0.000,0.000,0.795,0.795,0.789,0
4,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,5on4,...,1.0,1.000,1.0,0.019,0.000,0.000,0.224,0.224,0.224,0


Each row in the dataframe represents team statistics for a specific game. We can see that there are columns indicating the team, as well as the opposing team they faced in that game. There is also a unique gameID column that follows a standard format to identify each game separately. Each gameID has two separate sets of rows, one set for each team. However, there are multiple rows for each team/game combination. This is because there are rows for different game situations (listed in the 'situation' column), specifically around when one team or the other has a penalty, leaving them shorthanded. In fact, there are 5 rows for each team/game combination. For the purposes of predicting future game outcomes, we will only need the total statistics for each game, as it would be difficult to accurately predict the penalties that a team takes in a given match. As a result, we can filter the dataframe down to only include rows where the value in the situation column is "all".

In [7]:
# Filter mp_df to select only rows where 'situation' is equal to 'all'
filtered_mp_df = mp_df[mp_df['situation'] == 'all']

In [39]:
# Check to ensure that the new dataframe only contains rows where `situation` = 'all'
filtered_mp_df.head()

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,...,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame
1,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,all,...,31.0,30.369,5.0,0.396,0.168,0.168,2.917,2.833,2.714,0
6,NYR,2008,NYR,2008020003,NYR,T.B,HOME,20081005,Team Level,all,...,32.0,31.984,5.0,0.241,0.000,0.000,1.091,1.117,1.091,0
11,NYR,2008,NYR,2008020010,NYR,CHI,HOME,20081010,Team Level,all,...,45.0,43.911,3.0,0.448,0.407,0.407,2.738,2.751,2.730,0
16,NYR,2008,NYR,2008020019,NYR,PHI,AWAY,20081011,Team Level,all,...,41.0,38.025,4.0,0.504,0.401,0.401,3.123,2.958,2.907,0
21,NYR,2008,NYR,2008020034,NYR,N.J,HOME,20081013,Team Level,all,...,39.0,38.019,3.0,0.383,1.139,1.139,2.698,2.691,2.242,0


In [41]:
# Let's reset the index for now. We can drop it later if necessary.
filtered_mp_df.reset_index()

,index,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,...,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame
0,1,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,...,31.0,30.369,5.0,0.396,0.168,0.168,2.917,2.833,2.714,0
1,6,NYR,2008,NYR,2008020003,NYR,T.B,HOME,20081005,Team Level,...,32.0,31.984,5.0,0.241,0.000,0.000,1.091,1.117,1.091,0
2,11,NYR,2008,NYR,2008020010,NYR,CHI,HOME,20081010,Team Level,...,45.0,43.911,3.0,0.448,0.407,0.407,2.738,2.751,2.730,0
3,16,NYR,2008,NYR,2008020019,NYR,PHI,AWAY,20081011,Team Level,...,41.0,38.025,4.0,0.504,0.401,0.401,3.123,2.958,2.907,0
4,21,NYR,2008,NYR,2008020034,NYR,N.J,HOME,20081013,Team Level,...,39.0,38.019,3.0,0.383,1.139,1.139,2.698,2.691,2.242,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38055,190276,L.A,2015,L.A,2015030185,L.A,S.J,HOME,20160422,Team Level,...,36.0,39.463,10.0,0.370,0.261,0.261,2.152,2.314,2.307,1
38056,190281,L.A,2017,L.A,2017030171,L.A,VGK,AWAY,20180411,Team Level,...,51.0,52.184,4.0,0.509,0.000,0.000,2.788,2.783,2.733,1
38057,190286,L.A,2017,L.A,2017030172,L.A,VGK,AWAY,20180413,Team Level,...,76.0,75.118,3.0,0.788,0.962,0.962,4.469,4.347,4.234,1
38058,190291,L.A,2017,L.A,2017030173,L.A,VGK,HOME,20180415,Team Level,...,46.0,46.316,6.0,0.506,0.613,0.613,3.051,3.139,3.107,1


In [42]:
# What does the new dataframe look like?
filtered_mp_df.shape

(38060, 111)

In [9]:
print(f'There are {filtered_mp_df.shape[0]} columns and {filtered_mp_df.shape[1]} rows')

There are 38060 columns and 111 rows


Is this the actual number of games played over these seasons? Before I go and find the number of games per season (which has changed for multiple reasons, including strikes, lockouts, and COVID-19) and multiply by the number of teams (which has increased due to the addition of Las Vegas in 2016 and Seattle in 2019), I would like to take a look at the number of games per season, to ensure that it's consistent when expected. 

In [43]:
# Count the number of values per season
filtered_mp_df['season'].value_counts()

2021    2802
2022    2800
2018    2716
2017    2710
2013    2646
2015    2642
2014    2638
2009    2636
2010    2636
2016    2634
2011    2632
2008    2628
2019    2424
2020    1904
2012    1612
Name: season, dtype: int64

Oops, it's not consistent. For the majority of the time this dataset comprises, there were 30 teams, each playing 82 games. This works out to 2,460 games, which would be the amount we would expect to see for each season. There's only one possible explanation for the higher number - PLAYOFFS. And revisiting the dataframe info, we see that all the way at the end, the last column is called 'playoffGame', with a value of '1' when it is a playoff game. So this explains the inconsitency in yearly game totals, as the playoffs consist of four rounds of best-of-7 series, which means that the number of games varies depending on results. 

Should we include playoff games in our analysis? It is a commonly-held belief that the regular season is not an accurate predictor of playoff outcome, as the style of game and the officiating change in the playoffs. We will determine whether or not to include the playoffs before beginning our full analysis and modeling (unless there is time to do a separate analysis of the comparability of the regular season and the playoffs).

In [20]:
filtered_mp_df.isna().sum().sort_values(ascending=False)

team                               0
scoreVenueAdjustedxGoalsAgainst    0
playContinuedInZoneAgainst         0
playStoppedAgainst                 0
freezeAgainst                      0
                                  ..
playContinuedInZoneFor             0
playStoppedFor                     0
freezeFor                          0
reboundGoalsFor                    0
playoffGame                        0
Length: 111, dtype: int64

In [30]:
# Check for null values in each column
columns_with_null = filtered_mp_df.columns[filtered_mp_df.isna().any()]

# Display columns with null values, if any
if not columns_with_null.empty:
    print("Columns with null values:")
    print(columns_with_null)
else:
    print("No columns have null values.")

No columns have null values.


In [33]:
filtered_mp_df.duplicated().sum()

0

In [34]:
filtered_mp_df.T.duplicated().sum()

2

In [35]:
filtered_mp_df.T.duplicated()

team                                         False
season                                       False
name                                          True
gameId                                       False
playerTeam                                    True
                                             ...  
reboundxGoalsAgainst                         False
totalShotCreditAgainst                       False
scoreAdjustedTotalShotCreditAgainst          False
scoreFlurryAdjustedTotalShotCreditAgainst    False
playoffGame                                  False
Length: 111, dtype: bool

In [31]:
# Sort the results to see if there are more than two duplicate
duplicated_rows = filtered_mp_df.T.duplicated()
duplicated_rows.sort_values(ascending=False)

name                                      True
playerTeam                                True
team                                     False
flurryScoreVenueAdjustedxGoalsAgainst    False
playContinuedOutsideZoneAgainst          False
                                         ...  
savedShotsOnGoalFor                      False
playContinuedOutsideZoneFor              False
playContinuedInZoneFor                   False
playStoppedFor                           False
playoffGame                              False
Length: 111, dtype: bool

In [23]:
# top 5 rows showing only 'object' columns
filtered_mp_df.select_dtypes('object').head()

,team,name,playerTeam,opposingTeam,home_or_away,position,situation
1,NYR,NYR,NYR,T.B,AWAY,Team Level,all
6,NYR,NYR,NYR,T.B,HOME,Team Level,all
11,NYR,NYR,NYR,CHI,HOME,Team Level,all
16,NYR,NYR,NYR,PHI,AWAY,Team Level,all
21,NYR,NYR,NYR,N.J,HOME,Team Level,all


Now we can import our second dataset and examine it before merging them together. It contains the basic statistics for each game published by the NHL itself.

In [52]:
gts_df = pd.read_csv('game_teams_stats.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'game_teams_stats.csv'